In [7]:
import pandas as pd 
import numpy as np
from numpy import loadtxt
import warnings

from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import multiprocessing

from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.metrics import mean_squared_error as MSE
import matplotlib.pyplot as plt

In [8]:
realdata = pd.read_csv(f"C:\\Users\\namil\\Documents\\stmi-lab-namila\\allmetrics1")
savedval = 0.111

def basemodel(participant, arr, start, end, topredict, removesnacks, nodinners):
    global realdata
    global savedval
    
    test = realdata
    
    #remove specified entries
    if nodinners:
        test = test[test['Meal Type'] != "dinner"]
    
    if removesnacks:
        if(len(test[test['snack in 3hrs'] == True]) == 0):
            arr.append(savedval)
            print(f"RMSREcopy:{savedval}")
            return savedval
        else:
            test = test[test['snack in 3hrs'] == False]

    valid = test[test['dexcom 3hr auc'] != "error"]
    valid = valid[valid['mets 3 hr auc'] != "error"]
    
    X = valid.loc[:, start:end].to_numpy()
    #for predicting macros
    #X = valid.loc[:, 'peakheight':'mets 3 hr avg'].to_numpy()
    #for predicting ppgr 
    #X = valid.iloc[:, 10:18].to_numpy()
    #X = valid.loc[:, 'mets 3 hr auc':'fat'].to_numpy()
    #X = valid.loc[:, 'calories':'fat'].to_numpy()
    
    Y = valid[topredict].to_numpy().astype(float)
    warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)


    #split into train and test sets
    seed = 7
    test_size = 0.25
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
    
    # HP TUNING
    # Define the parameter grid
    param_grid = {'alpha': [0.0, 0.5, 1.0, 2.0],
                  'learning_rate': [0.3, 0.1, 0.01, 0.001],  # Learning rate of the model
                  'max_depth': [2, 3, 4, 5, 6],  # Maximum depth of each tree
                  'n_estimators': [100, 200, 250],  # Number of trees in the ensemble
                  }
    # Define the XGBoost regressor
    model = XGBRegressor()
    # Set up the grid search cross-validation
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
    # Perform the grid search
    grid_search.fit(X_train, y_train)
    # print(grid_search.best_params_)
    
    model = XGBRegressor(alpha = grid_search.best_params_['alpha'],
                        learning_rate = grid_search.best_params_['learning_rate'],
                        max_depth = grid_search.best_params_['max_depth'],
                        n_estimators = grid_search.best_params_['n_estimators'])
    #make model
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)

    plt.plot(y_test)
    plt.plot(prediction)
    plt.show()
    
#     # Fits the explainer
#     explainer = shap.Explainer(model.predict, X_test)
#     # Calculates the SHAP values - It takes some time
#     shap_values = explainer(X_train)
#     #shap.summary_plot(shap_values)
#     shap.plots.bar(shap_values)
    
    #sicong's normalized rmse
    errors = ((y_test - prediction)/y_test)**2
    rmsre = (np.average(errors))**.5
    print("RMSRE:", rmsre)
    
    arr.append(rmsre)
    return rmsre
    
def runfour(errors, start, end, topredict):
    global savedval
    savedval = basemodel(x, errors[0], start, end, topredict, 0, 1)
    basemodel(x, errors[1], start, end, topredict, 1, 1)
    savedval = basemodel(x, errors[2], start, end, topredict,  0, 0)
    basemodel(x, errors[3], start, end, topredict, 1, 0)

In [10]:
participants = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,26,27,28,29]
#participants = [26]
inputerrors = [[],[],[],[]]
inputerrors2 = [[],[],[],[]]
predpeakh = [[],[],[],[]]
predauc = [[],[],[],[]]
prediauc = [[],[],[],[]]
predpt = [[],[],[],[]]
predpd = [[],[],[],[]]
sbg = [[],[],[],[]]

for x in participants:
    print(x)
    
    global realdata
    realdata = pd.read_csv(f"C:\\Users\\namil\\Documents\\stmi-lab-namila\\allmetricsn{x}")
    
    #predicting macros
    start = 'peakheight'
    end = 'mets 3 hr avg'
    
    runfour(inputerrors, start, end, 'carbs')
    runfour(inputerrors2, start, end, 'calories')
        
    #predicting ppgr
    start = 'mets 3 hr auc'
    end = 'fat'

    runfour(predpeakh, start, end, 'peakheight')
    runfour(predauc, start, end, 'dexcom 3hr auc')
    runfour(prediauc, start, end, 'iauc')
    runfour(predpt, start, end, 'peaktime')
    runfour(predpd, start, end, 'peakduration_40')
    runfour(sbg, start, end, 'startbg')
    
df = pd.DataFrame({'participants': participants, 
                  'carbs bl raw rmsre': inputerrors[0], 'carbs bl removed rmsre': inputerrors[1],
                  'carbs bld raw rmsre': inputerrors[2], 'carbs bld removed rmsre': inputerrors[3],
                  'calories bl raw rmsre': inputerrors2[0], 'calories bl removed rmsre': inputerrors2[1],
                  'calories bld raw rmsre': inputerrors2[2], 'calories bld removed rmsre': inputerrors2[3],
                  'ph bl raw rmsre': predpeakh[0], 'ph bl removed rmsre': predpeakh[1],
                  'ph bld raw rmsre': predpeakh[2], 'ph bld removed rmsre': predpeakh[3],
                  'auc bl raw rmsre': predauc[0], 'auc bl removed rmsre': predauc[1],
                  'auc bld raw rmsre': predauc[2], 'auc bld removed rmsre': predauc[3],
                  'iauc bl raw rmsre': prediauc[0], 'iauc bl removed rmsre': prediauc[1],
                  'iauc bld raw rmsre': prediauc[2], 'iauc bld removed rmsre': prediauc[3],
                  'pt bl raw rmsre': predpt[0], 'pt bl removed rmsre': predpt[1],
                  'pt bld raw rmsre': predpt[2], 'pt bld removed rmsre': predpt[3],
                  'pd40 bl raw rmsre': predpd[0], 'pd40 bl removed rmsre': predpd[1],
                  'pd40 bld raw rmsre': predpd[2], 'pd40 bld removed rmsre': predpd[3],
                  'sbg bl raw rmsre': sbg[0], 'sbg bl removed rmsre': sbg[1],
                  'sbg bld raw rmsre': sbg[2], 'sbg bld removed rmsre': sbg[3]
                  })
df.to_csv("tunedactnohr")

#print(f"carbs raw error: {np.average(rawerrors)} \ncarbs removed error: {np.average(removederrors)} \ncarbs bld raw error: {np.average(drawerrors)}\ncarbs bld removed: {np.average(dremovederrors)}")
print("done!")

1
RMSRE: 0.33134954290950286
RMSRE: 1.10551400495811
RMSRE: 0.31703452410104993
RMSRE: 0.5966369234209534
RMSRE: 0.3562334130223793
RMSRE: 0.7324386309388023
RMSRE: 0.3550717835835433
RMSRE: 0.8176066371561984
RMSRE: 0.07155159207046084
RMSRE: 0.10337331343666169
RMSRE: 0.09772897662003471
RMSRE: 0.13419160323559137
RMSRE: 0.16880648991460795
RMSRE: 0.16705725317933554
RMSRE: 0.11176208695357338
RMSRE: 0.15934799425949892
RMSRE: 0.9891783498658072
RMSRE: 1.127345825147201
RMSRE: 0.8682737749819984
RMSRE: 0.9697616365772538
RMSRE: 0.4992166169838312
RMSRE: 0.5572419311046523
RMSRE: 0.41342618388008356
RMSRE: 1.6350419595285626
RMSRE: 0.5341597253542617
RMSRE: 0.8673860572909964
RMSRE: 0.49970927561752815
RMSRE: 0.5545455417526591
RMSRE: 0.14306836358990985
RMSRE: 0.16408864133599887
RMSRE: 0.1129024386883258
RMSRE: 0.1764066581836325
2
RMSRE: 0.4033440111589471
RMSRE: 1.6292771087125502
RMSRE: 0.37921153858122303
RMSRE: 0.9699128614916098
RMSRE: 0.451276265726576
RMSRE: 1.29100779189014

C:\Users\namil\AppData\Local\Temp\ipykernel_2252\2076690734.py:77: RuntimeWarning: divide by zero encountered in divide
  errors = ((y_test - prediction)/y_test)**2


RMSRE: inf
RMSREcopy:inf
RMSRE: 0.25962359687006514
RMSRE: 0.38127491527934754


C:\Users\namil\AppData\Local\Temp\ipykernel_2252\2076690734.py:77: RuntimeWarning: divide by zero encountered in divide
  errors = ((y_test - prediction)/y_test)**2


RMSRE: inf
RMSREcopy:inf
RMSRE: 0.5210226839437274
RMSRE: 0.41003993889424756
RMSRE: 0.2154617158079843
RMSREcopy:0.2154617158079843
RMSRE: 0.1447874739171103
RMSRE: 0.24578864989044485
6
RMSRE: 0.11727837395895385
RMSREcopy:0.11727837395895385
RMSRE: 0.8935480385492337
RMSRE: 1.0693886297835455
RMSRE: 0.25613215491932756
RMSREcopy:0.25613215491932756
RMSRE: 0.8112706550848942
RMSRE: 1.0575114756067674
RMSRE: 0.27557433172795026
RMSREcopy:0.27557433172795026
RMSRE: 0.18798868812506228
RMSRE: 0.18555274771642044
RMSRE: 0.18955039947951688
RMSREcopy:0.18955039947951688
RMSRE: 0.19164107644813239
RMSRE: 0.16024966976764013
RMSRE: 0.7953630909436024
RMSREcopy:0.7953630909436024
RMSRE: 0.8317156171798505
RMSRE: 1.0269766690745277
RMSRE: 0.6278308494030528
RMSREcopy:0.6278308494030528
RMSRE: 1.2369048387086832
RMSRE: 1.4101517311763965
RMSRE: 2.64827875862322
RMSREcopy:2.64827875862322
RMSRE: 0.7721762784725632
RMSRE: 0.6715620407207289
RMSRE: 0.10925382599051502
RMSREcopy:0.1092538259905150

...

C:\Users\namil\AppData\Local\Temp\ipykernel_2252\2076690734.py:77: RuntimeWarning: divide by zero encountered in divide
  errors = ((y_test - prediction)/y_test)**2


RMSRE: inf
RMSRE: 0.4426032409839806
RMSRE: 0.752882275046382
RMSRE: 0.743750621791003


C:\Users\namil\AppData\Local\Temp\ipykernel_2252\2076690734.py:77: RuntimeWarning: divide by zero encountered in divide
  errors = ((y_test - prediction)/y_test)**2


RMSRE: inf
RMSRE: 0.2793879994162839
RMSRE: 1.269036180522997
RMSRE: 4.793661766336024
RMSRE: 0.12598895141429534
RMSRE: 0.11412184673286532
RMSRE: 0.13046525462140654
RMSRE: 0.11288833908941473
18
RMSRE: 0.5525141115344205
RMSREcopy:0.5525141115344205
RMSRE: 0.6192979672379104
RMSRE: 1.441408006437267
RMSRE: 0.4152764998114839
RMSREcopy:0.4152764998114839
RMSRE: 0.8583744685455257
RMSRE: 0.9388857903188105
RMSRE: 0.2851216473679566
RMSREcopy:0.2851216473679566
RMSRE: 0.1475849915491925
RMSRE: 0.19111404536228974
RMSRE: 0.24678621934215494
RMSREcopy:0.24678621934215494
RMSRE: 0.22848091535386342
RMSRE: 0.22241774228795394
RMSRE: 3.713415699227734
RMSREcopy:3.713415699227734
RMSRE: 0.7644795906733077
RMSRE: 0.7823722957961924
RMSRE: 0.773081152316033
RMSREcopy:0.773081152316033
RMSRE: 0.6107448960212392
RMSRE: 1.1691697169092963
RMSRE: 0.7853724744720197
RMSREcopy:0.7853724744720197
RMSRE: 0.4616119069104742
RMSRE: 0.6813779214393544
RMSRE: 0.28159618801220365
RMSREcopy:0.28159618801220

RMSRE: 0.24717027671146125
RMSREcopy:0.24717027671146125
RMSRE: 0.17603708395025236
RMSREcopy:0.17603708395025236
RMSRE: 0.17029543070011002
RMSREcopy:0.17029543070011002
RMSRE: 0.46594647110841597
RMSREcopy:0.46594647110841597
RMSRE: 0.436596863706276
RMSREcopy:0.436596863706276
RMSRE: 0.21926744674710438
RMSREcopy:0.21926744674710438
RMSRE: 0.48220858911825576
RMSREcopy:0.48220858911825576
RMSRE: 0.8625681458246844
RMSREcopy:0.8625681458246844
RMSRE: 0.5053149735357891
RMSREcopy:0.5053149735357891
RMSRE: 0.1418937555433518
RMSREcopy:0.1418937555433518
RMSRE: 0.12397490243928579
RMSREcopy:0.12397490243928579
done!
